In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

sys.path.append('../src')
import cb_utils

sns.set(style="darkgrid")
pd.options.display.max_columns = 500

%load_ext autoreload
%autoreload 2

In [ ]:
# configuration
scoring_run_id = 1
use_cache = True
seed = 0
test_set_pct = 0.2

In [ ]:
# Load data
query = "SELECT * FROM cb.vw_mab_training_data WHERE mab_id = 5;"
data = cb_utils.sql_query_to_df(query, use_cache=use_cache)

In [ ]:
# limit to single assumption level

In [ ]:
feature_columns = [
    'lob_1_days'
  , 'lob_2_days'
  , 'lob_3_days'
  , 'grp_1_days'
  , 'grp_2_days'
  , 'grp_3_days'
  , 'grp_5_days'
  , 'grp_6_days'
  , 'grp_7_days'
  , 'grp_8_days'
  , 'unaligned_days'
#   , 'is_unaligned'
  , 'tc'
  , 'hcbs_tc'
  , 'icf_tc'
  , 'ip_tc'
  , 'rx_tc'
  , 'ed_tc'
  , 'snf_tc'
  , 'out_tc'
  , 'pro_tc'
  , 'spfac_tc'
  , 'amb_tc'
  , 'hh_tc'
  , 'hosp_tc'
  , 'oth_tc'
  , 'p_mm'
  , 'mm'
  , 'hcbs_respite_tc'
  , 'hcbs_fam_care_stip_tc'
  , 'hcbs_com_trans_tc'
  , 'hcbs_educ_train_tc'
  , 'hcbs_com_liv_fam_tc'
  , 'hcbs_com_liv_tc'
  , 'hcbs_attend_care_tc'
  , 'hcbs_com_trans_waiv_tc'
  , 'hcbs_home_meal_tc'
  , 'hcbs_pers_care_tc'
  , 'hcbs_ther_behav_tc'
  , 'hcbs_unsk_respite_tc'
  , 'hcbs_waiv_svc_tc'
  , 'ddos'
  , 'hcbs_ddos'
  , 'icf_ddos'
  , 'ip_ddos'
  , 'rx_ddos'
  , 'ed_ddos'
  , 'snf_ddos'
  , 'out_ddos'
  , 'pro_ddos'
  , 'spfac_ddos'
  , 'amb_ddos'
  , 'hh_ddos'
  , 'hosp_ddos'
  , 'oth_ddos'
  , 'pcp_ddos'
  , 'pulmonar_ddos'
  , 'cancer_ddos'
  , 'ckd_ddos'
  , 'esrd_ddos'
  , 'hyperlipid_ddos'
  , 'diab_ddos'
  , 'alzh_ddos'
  , 'dementia_ddos'
  , 'stroke_ddos'
  , 'hypertension_ddos'
  , 'fall_ddos'
  , 'transplant_ddos'
  , 'liver_ddos'
  , 'hippfract_ddos'
  , 'depression_ddos'
  , 'psychosis_ddos'
  , 'drug_ddos'
  , 'alcohol_ddos'
  , 'paralysis_ddos'
]
target_col = 'savings_tgt'
len(feature_columns)

### Try agg features at year and half year level

In [ ]:
# fully broken out month over month features
def features_mom(df, cols):
#     print('building month over month features')
    df = df.fillna(0)
    pre = df.query("period < 0")
    pre= pre.pivot(index='member_id', columns='period', values=cols)
    pre.columns = [f'{period}-{name}' for (name, period) in pre.columns]
    return pre.fillna(0)
features_mom.name = 'MOM'

In [ ]:
# agg semi yearly_features
def features_semi_annual(df, cols):
#     print('building semi annual features')
    df = df.fillna(0)
    pre = df.query("period < 0")
    h1 = pre.query('period < -6').groupby('member_id')
    h2 = pre.query('period >= -6').groupby('member_id')

    h1 = h1[cols].sum()
    h2 = h2[cols].sum()

    features_h1 = np.divide(h1[cols],  h1[['p_mm']])
    features_h2 = np.divide(h2[cols],  h2[['p_mm']])
    res = features_h2.merge(features_h1, left_index=True, right_index=True, suffixes=('_h2', '_h1'))
    return res.fillna(0)
features_semi_annual.name = 'Semi Annual'

In [ ]:
# agg yearly_features
def features_annual(df, cols):
#     print('building annual features')
    df = df.fillna(0)
    pre = df.query("period < 0").groupby('member_id')
    pre_sums = pre[cols].sum()
    res = np.divide(pre_sums[cols],  pre_sums[['p_mm']])
    return res.fillna(0)
features_annual.name = 'Annual'

In [ ]:
def print_feature_importance(regr, cols, max_cols=20):
    print('Feature Importance')
    i = 0
    for imp, feat in sorted([(b, a) for a, b in zip(cols, regr.feature_importances_)], reverse=True):
        if imp > 0.001:
            print('%0.3f: %s' % (imp, feat))
            i += 1
        if i > max_cols:
            break

In [ ]:
def train_and_test(feats_df, targets_df):
    # order features and targets by member id, make sure they line up perfectly
    feats_df.sort_index(inplace=True)
    targets_df.sort_index(inplace=True)
    assert sum(targets_df.index - feats_df.index) == 0

    # train test split
    target_col = 'savings_tgt'
    X_train, X_test, y_train, y_test = train_test_split(feats_df, targets_df[target_col], test_size=test_set_pct, random_state=seed)

    gb = GradientBoostingRegressor(random_state=seed)
#     gb = RandomForestRegressor(random_state=seed)

    gb.fit(X_train, y_train)
    verbose = True
    preds = gb.predict(X_test)
    r2_score = gb.score(X_test, y_test)
    error = np.abs(y_test - preds)
    mean_hrs_error = error.mean()
    median_hrs_error = error.median()
    

    print(f'R^2 Score: {r2_score}')
    print(f'Mean absolute $ error: {mean_hrs_error}')
    print(f'Median absolute $ error: {median_hrs_error}')
    print_feature_importance(gb, features_df.columns)

    if verbose:
        fig, ax = plt.subplots(nrows=1, figsize=(20,10))
        ax.scatter(preds, y_test)
        ax.set_xlabel('preds')
        ax.set_ylabel('actual');
        ax.set_title(f'Predicted vs actual savings')
        plt.show()
    
    return gb

In [ ]:
search_level_feature_space = False
if search_level_feature_space:
    levels = ('1. h', '2. m', '3. l')
    feature_funcs = (features_mom, features_semi_annual, features_annual)
    for lvl in levels:
        df = data.loc[data.lvl_tgt == lvl]

        targets_df = df[['member_id', target_col]].groupby('member_id').first()
        targets_df.fillna(0, inplace=True)

        for feat_func in feature_funcs: 
            print('Level: ', lvl)
            print('Features: ', feat_func.name)
            df = df.fillna(0)
            features_df = feat_func(df, feature_columns)
            model = train_and_test(features_df, targets_df)


In [ ]:
df = data.loc[data.lvl_tgt == '1. h']

targets_df = df[['member_id', target_col]].groupby('member_id').first()
targets_df.fillna(0, inplace=True)

df = df.fillna(0)
features_df = features_annual(df, feature_columns)
model = train_and_test(features_df, targets_df)

In [ ]:
preds = model.predict(features_df)
top_500_preds = features_df.assign(pred=preds).sort_values('pred', ascending=False).iloc[:500][['pred']]
id_pop = data.loc[(data.lvl_tgt == '1. h')]
# id_pop = data.loc[(data.lvl_tgt == lvl) & (data.period > 0)]
id_pop = id_pop.set_index('member_id')
id_pop = id_pop.merge(top_500_preds, left_index=True, right_index=True) 
id_pop = id_pop.merge(targets_df, left_index=True, right_index=True) 
id_pop = id_pop.fillna(0)

In [ ]:
id_pop.head()
# features_df.assign(pred=preds)[['pred']].to_csv('../outputs/h_preds.csv')

In [ ]:
tcs = [c for c in features_df.columns if c[-3:] == '_tc'] + ['tc']
tcs = [c for c in tcs if len(c) < 8 or c[:5] != 'hcbs_']

g = sns.relplot(
    x="period",
    y="value",
    hue="variable",
    kind="line",
    data=id_pop.melt(id_vars=['period'], value_vars=tcs),
    height=12,
    aspect=2
)

In [ ]:
# tune hyperparameters: didn't see much benifit, takes about 30 minutes to run

# lvl  = '1. h'
# df = data.loc[data.lvl_tgt == lvl]
# df = df.fillna(0)

# targets_df = df[['member_id', target_col]].groupby('member_id').first()
# feats_df = features_annual(df, feature_columns)

# feats_df.sort_index(inplace=True)
# targets_df.sort_index(inplace=True)
# assert sum(targets_df.index - feats_df.index) == 0

# # train test split
# target_col = 'savings_tgt'
# X_train, X_test, y_train, y_test = train_test_split(feats_df, targets_df[target_col], test_size=test_set_pct, random_state=seed)

# gb = GradientBoostingRegressor(random_state=seed)

# grid = {
#     'learning_rate': [0.001, 0.01, 0.1, 0.2],
#     'n_estimators': [10, 100, 500, 1000],
#     'max_depth': [2, 3, 4, 5, 10],
#     'min_samples_leaf': [1, 2, 3, 4, 5],
# }

# clf = GridSearchCV(gb, grid, n_jobs=os.cpu_count())

# search = clf.fit(X_train, y_train)

# cv_res = pd.DataFrame(search.cv_results_)
# cv_res.sort_values('rank_test_score')

In [ ]:
# model_name = cb_utils.save_model(model, '1_xgb_cat_savings', {'name': '1_xgb_cat_savings', 'features': list(features_df.columns)})

In [ ]:
# load model trained on mab 5
model_name = '20200731_153512_1_xgb_cat_savings'
model, meta = cb_utils.load_model(model_name)

In [ ]:
# cb_utils.save_scores(final)

In [ ]:
# test on a different population
query = "SELECT * FROM cb.vw_mab_training_data WHERE mab_id = 6;"
mab6 = cb_utils.sql_query_to_df(query, use_cache=use_cache)

In [ ]:
targets_df = mab6[['member_id', target_col]].groupby('member_id').first()
targets_df.fillna(0, inplace=True)

mab6 = mab6.fillna(0)
features_df = features_annual(mab6, feature_columns)
features_df.sort_index(inplace=True)
targets_df.sort_index(inplace=True)


preds = model.predict(features_df)
r2_score = model.score(features_df, targets_df[target_col])

error = np.abs(targets_df[target_col] - preds)
mean_hrs_error = error.mean()
median_hrs_error = error.median()

print('Testing MAB 6 data on model trained with mab 5')
print(f'R^2 Score: {r2_score}')
print(f'Mean absolute $ error: {mean_hrs_error}')
print(f'Median absolute $ error: {median_hrs_error}')

fig, ax = plt.subplots(nrows=1, figsize=(20,10))
ax.scatter(preds, targets_df[target_col])
ax.set_xlabel('preds')
ax.set_ylabel('actual');
ax.set_title(f'Predicted vs actual savings')
plt.show()


In [ ]:
# try training on early set 
query = "SELECT * FROM cb.vw_mab_training_data WHERE mab_id = 6;"
mab6 = cb_utils.sql_query_to_df(query, use_cache=use_cache)

df = mab6.loc[data.lvl_tgt == '1. h']

targets_df = df[['member_id', target_col]].groupby('member_id').first()
targets_df.fillna(0, inplace=True)

df = df.fillna(0)
features_df = features_annual(df, feature_columns)
model = train_and_test(features_df, targets_df)